In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sns
import param_generation as pg
import helpers as hp
from ipywidgets import GridspecLayout,HBox,widgets
from IPython.display import Audio, display
import librosa
import random
import common_vars as commons
import glob


#let's play the sound given the parameters
import imp
imp.reload(pg)
imp.reload(hp)
imp.reload(commons)

sr=41000
classes=commons.classes
classes_ranked=[c+"_rank" for c in classes]
param_cols=['oscType_0',
       'isNoise_0', 'A_0', 'D_0', 'S_0', 'R_0', 'pitch_0_0', 'pitch_1_0',
       'pitch_2_0', 'pitch_3_0', 'bpCutLow_0', 'bpCutHigh_0',
       'bpOrder_0', 'amplitude_0','start_0','length_0']

pc_no_num=[x[0:-2] for x in param_cols] # param columns with no _# 


# df = pd.read_csv("csvs/param_scores.csv",error_bad_lines=False)
df = pd.concat([pd.read_csv(f) for f in glob.glob('csvs/*.csv')], ignore_index = True)

df.columns=[*classes_ranked,*classes,*param_cols]
# df["oscType_0"]=df["oscType_0"].replace(dict(zip(["sine","square","saw"],[0,1,2])))
df=df.fillna(0)
pd.set_option('display.max_columns', 33)


ValueError: No objects to concatenate

In [3]:
df[classes].describe().loc[["mean","std","25%","75%"]]

NameError: name 'df' is not defined

In [ ]:
df[classes_ranked].describe().loc[["mean","std","50%"]]
df

In [4]:
#visualizing ranks per group
gc=pd.DataFrame(columns=range(1,5))
for c in classes_ranked:
    gc=gc.append(df[c].astype(int).value_counts())

gc=gc.sort_values(by=[1,2],ascending=False)
fig = plt.gcf()
fig.set_size_inches(15, 7.5)

gc[list(range(1,6))].plot(kind="bar",figsize=(15,4),stacked=True)
plt.legend(fontsize=20) # using a size in point
plt.xticks(fontsize=15, rotation=45)
gc[list(range(6,10))].plot(kind="bar",figsize=(15,4),stacked=True)

NameError: name 'df' is not defined

In [5]:
import imp
imp.reload(pg)
imp.reload(hp)

#Let's hear some of the "best" generated sounds for each group, set g to "sound group" of choice
g="snare"
selection=df[(df[g]>0.2) & (df[g+"_rank"]<3) & (df["synth_rank"]>1)].reset_index(drop=True)
print(selection.shape)

x,y=4,4
grid = GridspecLayout(x, y)
for i in range(x):
    for j in range(y):
        r=selection.iloc[random.randint(0,selection.shape[0]-1)]
        r=r[param_cols]
        pset=hp.rToParams(r)
        out=pg.ensemble([pset])
        o= widgets.Output()
        with o:
            display(Audio(hp.memToAud(out),rate=sr,autoplay=False))
        grid[i, j] = o
display(grid)


NameError: name 'df' is not defined

In [6]:
#constructing multivariate distributions to sample from
#get params, get cov matrix, construct distribution
v=selection[param_cols].to_numpy().T
#convert oscType column to 
cov=np.cov(v)
mu=v.mean(1)
#average
pd.DataFrame([mu],columns=param_cols)
selection[param_cols].describe()

NameError: name 'selection' is not defined

In [7]:
#you can see this approach isn't amazing

def randomSample(t=0):
    p=np.random.multivariate_normal(mu,cov)
    p_dataframed=pd.DataFrame([p],columns=param_cols)
    p_series=p_dataframed.iloc[0][param_cols]
    pset=hp.rToParams(p_series)
    o=pg.ensemble([pset])
    o=librosa.util.normalize(hp.memToAud(o))
    return o,pset,t
o,pset,t=randomSample()
Audio(o,rate=sr,autoplay=True)

NameError: name 'mu' is not defined

In [10]:
#let's try evolution strategys with deap
import scorer
import imp
imp.reload(scorer)
imp.reload(pg)
imp.reload(hp)
from deap import base, creator
import random
from deap import tools
import scipy.spatial.distance as d
from helpers import rToParams

toolbox = base.Toolbox()

creator.create("FitnessMin", base.Fitness, weights=(-1,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def mutateArraySearch(indiv,a):
    e=0.5
    t=0.5
    r=pd.DataFrame([a],columns=param_cols)
    pset=rToParams(r.iloc[0])
    pset.mutate(e,t)
    a_m=list(vars(pset).values())
    return indiv(a_m)

drum="kick"
goodGenes=df[(df[g]>0.15) & (df[g+"_rank"]<3) & (df["synth_rank"]>1)].reset_index(drop=True)

def initPopulation(pcls,ng):
    ipop=[creator.Individual(goodGenes[param_cols].iloc[random.randint(0,goodGenes.shape[0])].values)
          for x in range(ng)]
    return pcls(ipop)

def evaluate(individual):
#     distance=d.minkowski(individual,solution) # distance between params and the "ideal drum"
    ind_df=pd.DataFrame([individual],columns=param_cols)
    p_series=ind_df.iloc[0]

    scores,valid=scorer.rank_score(p_series)
    
    if valid:
        rank=scores["%s_rank"%(drum,)]
        score=scores["%s"%(drum,)]
    else:
        rank=12
        score=0
    f=rank-score
#     print (f,rank,score,"\r",end="")
    return (f,)

def randSelect(l,k):
    return random.sample(l, k)
# toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list,)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("param_mutate",mutateArraySearch,creator.Individual)
toolbox.register("select_best", tools.selBest,fit_attr='fitness')
toolbox.register("select_random",randSelect)
toolbox.register("evaluate", evaluate)
hof=tools.HallOfFame(POP_SIZE*2)

ImportError: cannot import name 'CNN_utils' from 'feature_extraction' (/home/asalimi/Synths_Stacks_Search/feature_extraction/__init__.py)

In [9]:
%%time
CXPB, MUTPB = 0.5, 1 #making babies,mutation prob,num generations
MUTENV,MUTTEXT=0.3,0.2
POP_SIZE = 20
hof=tools.HallOfFame(POP_SIZE*2)

def evolve(pop,hof):
        newGenes=toolbox.population_guess(int(0.2*POP_SIZE))
        offspring = pop.copy() + newGenes
        # Clone the selected individuals        
        for i,mutant in enumerate(offspring):
            if random.random() < MUTPB:
#                 print("before mut",offspring[i])
                offspring[i]=toolbox.param_mutate(mutant)
#                 print("after mut ",offspring[i])
                del offspring[i].fitness.values
                
        # Apply crossover and mutation on the offspring
        for child1, child2 in list(zip(offspring[::2], offspring[1::2])):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        #update hof

        offspring = toolbox.select_best(offspring,POP_SIZE)
        
        hof.update(offspring)
        return offspring

NameError: name 'tools' is not defined

In [764]:
NGEN=30
def main():
#     pop = toolbox.population_guess(POP_SIZE)
    pop=hof.items
    hof.update(pop)
    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    for g in range(NGEN):
        pop=evolve(pop,hof)
        print (g,"\r",end="")
    return pop
pop=main()

In [2]:
def pl(l):
    for p in l:
        print(p,p.fitness)
# pl(pop)
# pop=evolve(pop)
# print("new")
# pl(pop)
# len(pop)
pl(hof)

NameError: name 'hof' is not defined

In [3]:
#how does mutation work?

# toolbox.select_random(pop,POP_SIZE//2)
pop=toolbox.population_guess(POP_SIZE)

NameError: name 'toolbox' is not defined

In [4]:
c1,c2=pop[0],pop[2]
# Select the next generation individuals
toolbox.select(pop, 5)
# # Clone the selected individuals
# offspring = list(map(toolbox.clone, offspring))
# # Apply crossover and mutation on the offspring
# for child1, child2 in list(zip(offspring[::2], offspring[1::2])):
#     if random.random() < CXPB:
#         toolbox.mate(child1, child2)
#         del child1.fitness.values
#         del child2.fitness.values
# print(c1,c2)
# print(tools.cxTwoPoint(c1,c2,))

NameError: name 'pop' is not defined

In [765]:
# What did our DAEP search return?

z=hof.items.copy()
# z=pop
def mutateArray(a,e=0.5,t=0.5):
    r=pd.DataFrame([a],columns=param_cols)
    pset=rToParams(r.iloc[0])
    pset.mutate(e,t)
    return pset


x,y=4,int((len(z)/4))
grid=[]
grid = GridspecLayout(x, y)
for i in range(x):
    for j in range(y):
        o= widgets.Output()
        sample_number=i+j+(i*(y-1))
        pset=mutateArray(z[sample_number],e=0,t=0)
#         print()
        try:
            out=pg.ensemble([pset])
            with o:
                display(Audio(hp.memToAud(out),rate=sr,autoplay=False,embed=True),evaluate(z[sample_number]))
            grid[i, j] = o
        except:
            continue
display(grid)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…